In [433]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, LSTM, BatchNormalization, Dropout, RepeatVector, TimeDistributed, concatenate
from tensorflow.keras.optimizers import Adam
from scipy import stats
from sklearn.model_selection import train_test_split

In [413]:
VehicleDF = pd.read_excel('Dataset.xlsx', sheet_name='Trip')
ChargingDF = pd.read_excel('Dataset.xlsx', sheet_name='Charge Cycle')

### Preprocessing

In [414]:
ChargingDF['Local Charge Start Time'] = pd.to_datetime(ChargingDF['Local Charge Start Time'],
                                                       format='mixed')
ChargingDF['Local Charge End Time'] = pd.to_datetime(ChargingDF['Local Charge End Time'],
                                                     format='mixed')
VehicleDF['Local Trip Start Time'] = pd.to_datetime(VehicleDF['Local Trip Start Time'],
                                                    format='mixed')
VehicleDF['Local Trip End Time'] = pd.to_datetime(VehicleDF['Local Trip End Time'],
                                                  format='mixed')

In [415]:
characteristics = pd.concat([ChargingDF.isnull().sum(), 
                             ChargingDF.notnull().sum(), 
                             ChargingDF.nunique(), 
                             ChargingDF.dtypes],
                        keys=['Null Values', 
                              'Available Values', 
                              'Unique Values', 
                              'Data Types'],
                        axis=1)
characteristics

,Null Values,Available Values,Unique Values,Data Types
Vehicle ID,0,6296,8,object
Local Charge Start Time,0,6296,6274,datetime64[ns]
Local Charge End Time,0,6296,6265,datetime64[ns]
Starting SOC,0,6296,170,float64
Ending SOC,0,6296,120,float64
Total Energy Delivered,0,6296,5705,float64
Average Power,0,6296,6210,float64
Max Power,0,6296,4371,float64


In [416]:
characteristics = pd.concat([VehicleDF.isnull().sum(), 
                             VehicleDF.notnull().sum(), 
                             VehicleDF.nunique(), 
                             VehicleDF.dtypes],
                        keys=['Null Values', 
                              'Available Values', 
                              'Unique Values', 
                              'Data Types'],
                        axis=1)
characteristics

,Null Values,Available Values,Unique Values,Data Types
Vehicle ID,0,8013,27,object
Local Trip Start Time,0,8013,7893,datetime64[ns]
Local Trip End Time,0,8013,7931,datetime64[ns]
Initial SOC,0,8013,3161,float64
Final SOC,0,8013,3257,float64
SOC Used,0,8013,3271,float64
Total Energy Consumption,0,8013,7850,float64
Total Distance,0,8013,5465,float64
Average Ambient Temperature,0,8013,4849,float64


In [417]:
ChargingDF['Charging Duration (Hours)'] = round((
    ChargingDF['Local Charge End Time'] - ChargingDF['Local Charge Start Time']
    ).dt.total_seconds() / 3600, 2)
ChargingDF['Total SOC Charged'] = (
    ChargingDF['Ending SOC'] - ChargingDF['Starting SOC']
    )
ChargingDF.head(5)

,Vehicle ID,Local Charge Start Time,Local Charge End Time,Starting SOC,Ending SOC,Total Energy Delivered,Average Power,Max Power,Charging Duration (Hours),Total SOC Charged
0,EV026,2017-09-01 08:05:00,2017-09-01 08:09:00,81.0,99.5,13.910,193.386,342.28,0.07,18.5
1,EV026,2017-09-01 09:02:00,2017-09-01 09:11:00,65.0,100.0,24.091,308.800,363.64,0.15,35.0
2,EV026,2017-09-01 10:08:00,2017-09-01 10:14:00,57.5,100.0,31.054,327.830,364.82,0.10,42.5
3,EV026,2017-09-01 11:05:00,2017-09-01 11:14:00,62.5,99.5,25.935,321.533,361.06,0.15,37.0
4,EV026,2017-09-01 12:10:00,2017-09-01 12:19:00,56.5,100.0,32.010,230.942,360.10,0.15,43.5


In [418]:
VehicleDF['Energy Efficiency'] = np.where(VehicleDF['Total Distance'] == 0, 
                                          np.nan, 
                                          round(VehicleDF['SOC Used'] 
                                                / VehicleDF['Total Distance'], 2))
VehicleDF['Trip Duration (Hours)'] = (
    VehicleDF['Local Trip End Time'] - VehicleDF['Local Trip Start Time']
    ).dt.total_seconds() / 3600
VehicleDF = round(VehicleDF,3)

In [419]:
VehicleDF = VehicleDF.drop(labels=['Local Trip Start Time'], axis=1)

In [420]:
VehicleDF = VehicleDF[VehicleDF['Trip Duration (Hours)'] >= 0]

In [421]:
VehicleDF.sort_values(by=['Vehicle ID', 'Local Trip End Time'], inplace=True)
VehicleDF.set_index('Local Trip End Time', inplace=True)

In [422]:
VehicleDF

,Vehicle ID,Initial SOC,Final SOC,SOC Used,Total Energy Consumption,Total Distance,Average Ambient Temperature,Energy Efficiency,Trip Duration (Hours)
Local Trip End Time,,,,,,,,,
2018-01-10 19:00:00,EV026,100.000,47.000,53.000,36.609,18.991,56.840,2.79,1.783
2018-01-10 21:13:00,EV026,59.500,77.500,-18.000,26.207,14.488,54.680,-1.24,1.217
2018-01-11 19:05:00,EV026,76.500,45.500,31.000,21.004,11.481,55.760,2.70,0.917
2018-01-11 20:25:00,EV026,99.000,52.500,46.500,25.429,13.375,53.240,3.48,1.083
2018-01-12 13:48:00,EV026,60.000,53.000,7.000,24.238,10.213,52.880,0.69,1.300
...,...,...,...,...,...,...,...,...,...
2022-11-21 08:52:00,EV169,56.022,52.016,7.853,10.178,3.607,66.740,2.18,2.383
2023-01-23 12:24:00,EV169,46.167,45.119,1.166,4.670,0.121,39.175,9.62,0.129
2023-01-23 13:29:00,EV169,44.572,43.139,1.467,5.032,0.068,48.523,21.47,0.283


In [423]:
ChargingDF.head(5)

,Vehicle ID,Local Charge Start Time,Local Charge End Time,Starting SOC,Ending SOC,Total Energy Delivered,Average Power,Max Power,Charging Duration (Hours),Total SOC Charged
0,EV026,2017-09-01 08:05:00,2017-09-01 08:09:00,81.0,99.5,13.910,193.386,342.28,0.07,18.5
1,EV026,2017-09-01 09:02:00,2017-09-01 09:11:00,65.0,100.0,24.091,308.800,363.64,0.15,35.0
2,EV026,2017-09-01 10:08:00,2017-09-01 10:14:00,57.5,100.0,31.054,327.830,364.82,0.10,42.5
3,EV026,2017-09-01 11:05:00,2017-09-01 11:14:00,62.5,99.5,25.935,321.533,361.06,0.15,37.0
4,EV026,2017-09-01 12:10:00,2017-09-01 12:19:00,56.5,100.0,32.010,230.942,360.10,0.15,43.5


In [424]:
ChargingDF.drop('Local Charge Start Time', axis=1, inplace=True)

In [425]:
ChargingDF.sort_values(by=['Local Charge End Time', 'Vehicle ID'], inplace=True)
ChargingDF.set_index('Local Charge End Time', inplace=True)

In [426]:
ChargingDF.head(5)

,Vehicle ID,Starting SOC,Ending SOC,Total Energy Delivered,Average Power,Max Power,Charging Duration (Hours),Total SOC Charged
Local Charge End Time,,,,,,,,
2017-09-01 08:09:00,EV026,81.0,99.5,13.910,193.386,342.28,0.07,18.5
2017-09-01 09:11:00,EV026,65.0,100.0,24.091,308.800,363.64,0.15,35.0
2017-09-01 10:14:00,EV026,57.5,100.0,31.054,327.830,364.82,0.10,42.5
2017-09-01 11:14:00,EV026,62.5,99.5,25.935,321.533,361.06,0.15,37.0
2017-09-01 12:19:00,EV026,56.5,100.0,32.010,230.942,360.10,0.15,43.5


In [427]:
#Removing outliers from the Datasets.

NumericalColumnsC = ['Starting SOC', 'Ending SOC', 'Total Energy Delivered',
                     'Average Power', 'Max Power', 'Charging Duration (Hours)',
                      'Total SOC Charged']

ZScoreC = stats.zscore(ChargingDF[NumericalColumnsC])

FilteredChargingDF = ChargingDF[(ZScoreC < 3).all(axis=1)]

print(f"Number of Rows before filtering: {len(ChargingDF)}")
print(f"Number of Rows after Z Score filtering: {len(FilteredChargingDF)}")

Number of Rows before filtering: 6296
Number of Rows after Z Score filtering: 6189


In [428]:
ChargingDF = FilteredChargingDF.copy()

In [429]:
round(ChargingDF[NumericalColumnsC].describe(), 2)

,Starting SOC,Ending SOC,Total Energy Delivered,Average Power,Max Power,Charging Duration (Hours),Total SOC Charged
count,6189.00,6189.00,6189.00,6189.00,6189.00,6189.00,6189.00
mean,59.77,93.23,23.93,232.73,305.92,0.11,33.46
std,12.22,9.76,9.20,68.88,68.97,0.05,12.19
min,9.50,33.50,0.00,-16.56,-16.36,0.00,-1.50
25%,53.00,90.50,19.20,197.77,275.24,0.08,26.50
50%,58.00,97.50,24.99,243.26,326.14,0.12,35.50
75%,66.00,100.00,30.46,282.58,356.66,0.15,42.00
max,98.00,100.00,52.38,367.30,382.24,0.50,70.50


In [430]:
NumericalColumnsT = ['Initial SOC', 'Final SOC', 'SOC Used',
                     'Total Energy Consumption','Total Distance',
                     'Average Ambient Temperature', 'Energy Efficiency',
                     'Trip Duration (Hours)']

Q1 = VehicleDF[NumericalColumnsT].quantile(0.15)
Q3 = VehicleDF[NumericalColumnsT].quantile(0.85)
InterQuartileRange = Q3 - Q1


LowerBound = Q1 - 1.5 * InterQuartileRange
UpperBound = Q3 - 1.5 * InterQuartileRange

FilteredVehicleDF = VehicleDF[
    ~((VehicleDF[NumericalColumnsT] >= LowerBound) &
      (VehicleDF[NumericalColumnsT] <= UpperBound)).any(axis=1)
]

print(f"Number of Rows before filtering: {len(VehicleDF)}")
print(f"Number of Rows after IQR filtering: {len(FilteredVehicleDF)}")

Number of Rows before filtering: 8009
Number of Rows after IQR filtering: 7159


In [431]:
VehicleDF = FilteredVehicleDF.copy()

In [432]:
round(VehicleDF[NumericalColumnsT].describe(), 2)

,Initial SOC,Final SOC,SOC Used,Total Energy Consumption,Total Distance,Average Ambient Temperature,Energy Efficiency,Trip Duration (Hours)
count,7159.00,7159.00,7159.00,7159.00,7159.00,7159.00,6168.00,7159.00
mean,90.13,78.37,18.29,55.50,23.04,70.93,2.90,3.12
std,11.63,17.64,22.08,81.72,32.77,13.57,9.56,4.50
min,0.00,32.00,-20.50,0.00,0.00,7.25,-44.81,0.02
25%,84.50,62.50,0.33,1.29,0.16,61.88,0.52,0.38
50%,94.50,83.00,6.50,25.12,12.15,71.60,0.90,1.17
75%,99.00,93.94,35.50,72.58,32.36,79.78,2.73,3.72
max,100.00,100.00,187.87,533.48,181.77,108.68,425.81,24.15


### Data Preparation

In [442]:
from sklearn.preprocessing import StandardScaler, LabelBinarizer

FeaturesT = ['Initial SOC', 
             'Final SOC', 
             'Total Energy Consumption', 
             'Total Distance', 
             'Average Ambient Temperature']
TargetT = ['SOC Used']

FeaturesC = ['Starting SOC', 
             'Ending SOC', 
             'Total Energy Delivered', 
             'Average Power', 
             'Max Power',
             'Charging Duration (Hours)']
TargetC = ['Total SOC Charged']


ScalerTrip = StandardScaler()
VehicleDF[FeaturesT] = ScalerTrip.fit_transform(VehicleDF[FeaturesT])
VehicleDF[TargetT] = ScalerTrip.fit_transform(VehicleDF[TargetT])

ScalerCharge = StandardScaler()
ChargingDF[FeaturesC] = ScalerCharge.fit_transform(ChargingDF[FeaturesC])
ChargingDF[TargetC] = ScalerCharge.fit_transform(ChargingDF[TargetC])


In [435]:
Encoder = LabelBinarizer()

VEncoded = Encoder.fit_transform(VehicleDF['Vehicle ID'])

EncodedTVID = [f"Vehicle_{VClass}" for VClass in Encoder.classes_]
EncodedTDF = pd.DataFrame(VEncoded, columns=EncodedTVID)
VehicleDF = pd.concat([VehicleDF.reset_index(drop=True),
                       EncodedTDF.reset_index(drop=True)],
                       axis=1)

CEncoded = Encoder.fit_transform(ChargingDF['Vehicle ID'])

EncodedCVID = [f"Vehicle_{VClass}" for VClass in Encoder.classes_]
EncodedCDF = pd.DataFrame(CEncoded, columns=EncodedCVID)
ChargingDF = pd.concat([ChargingDF.reset_index(drop=True),
                        EncodedCDF.reset_index(drop=True)],
                        axis=1)

In [445]:
def CreateSequences(Data, VehicleIDCol, EncodedVIDCols, Features, Target, SequenceLength):
    """
    Create sequences of Features & Targets grouped by Vehicle ID.

    Parameters:
    - Data: DataFrame containing the data.
    - VehicleCol: Column name for Vehicle IDs.
    - EncodedVIDCols: List one-hot encoded Vehicle ID column names.
    - Features: List of feature column names.
    - Target: Target column name.
    - n: number of time steps in each sequence.

    Returns:
    - X: Numpy array of input sequences (shape: NumSampels, SequenceLength, NumFeatures)
    - y: Numpy array of target values (shape: NumSamples, 1)
    """

    X, y = [], []

    Data = Data.reset_index(drop=True)

    Grouped  = Data.groupby(VehicleIDCol)
    
    for VehicleID, Group in Grouped:

        Group = Group.sort_index()

        for i in range(len(Group) - SequenceLength):
            FeaturesCols = Features + EncodedVIDCols
            X.append(Group[FeaturesCols].iloc[i:i+SequenceLength].values)
            y.append(Group[Target].iloc[i+SequenceLength].values)

    return np.array(X), np.array(y)

SequenceLength = 5

XTrip, yTrip = CreateSequences(
    Data=VehicleDF,
    VehicleIDCol='Vehicle ID',
    Features=FeaturesT,
    EncodedVIDCols=EncodedTVID,
    Target=TargetT,
    SequenceLength=SequenceLength,
)

XCharge, yCharge = CreateSequences(
    Data=ChargingDF,
    VehicleIDCol='Vehicle ID',
    EncodedVIDCols=EncodedCVID,
    Features=FeaturesC,
    Target=TargetC,
    SequenceLength=SequenceLength
)

print("XTrip shape: ", XTrip.shape)
print("yTrip shape: ", yTrip.shape)

print("\nXCharge shape: ", XCharge.shape)
print("yCharge shape: ", yCharge.shape)

XTrip shape:  (7024, 5, 32)
yTrip shape:  (7024, 1)

XCharge shape:  (6149, 5, 14)
yCharge shape:  (6149, 1)


In [453]:
def DataPrep(X1, X2, y1, y2, TestSize=0.3, RandomState=301):
    

    MinSamples = min(len(X1), len(X2), len(y1), len(y2))

    X1, y1 = X1[:MinSamples], y1[:MinSamples]
    X2, y2 = X2[:MinSamples], y2[:MinSamples]

    y1 = np.expand_dims(y1, axis=-1)
    y2 = np.expand_dims(y2, axis=-1)
    

    X1Train, X1Temp, y1Train, y1Temp = train_test_split(X1, y1,
                                                    test_size=TestSize,
                                                    random_state=RandomState)
    
    X2Train, X2Temp, y2Train, y2Temp = train_test_split(X2, y2,
                                                    test_size=TestSize,
                                                    random_state=RandomState)
    
    X1Val, X1Test, y1Val, y1Test = train_test_split(X1Temp, y1Temp,
                                                    test_size=0.5,
                                                    random_state=RandomState)
    
    X2Val, X2Test, y2Val, y2Test = train_test_split(X2Temp, y2Temp,
                                                    test_size=0.5,
                                                    random_state=RandomState)
    
    return (X1Train, X1Val, X1Test, y1Train, y1Val, y1Test), \
        (X2Train, X2Val, X2Test, y2Train, y2Val, y2Test)


(TripTrain, TripVal, TripTest, SOCTrain, SOCVal, SOCTest), \
(ChargeTrain, ChargeVal, ChargeTest, AnomalyTrain, AnomalyVal, AnomalyTest) = DataPrep(
    XTrip, XCharge, yTrip, yCharge
    )

In [454]:
print("TripTrain shape:", TripTrain.shape)
print("ChargeTrain shape:", ChargeTrain.shape)
print("SOCTrain shape:", SOCTrain.shape)
print("AnomalyTrain shape:", AnomalyTrain.shape)

print("\nTripVal shape:", TripVal.shape)
print("ChargeVal shape:", ChargeVal.shape)
print("SOCVal shape:", SOCVal.shape)
print("AnomalyVal shape:", AnomalyVal.shape)

TripTrain shape: (4304, 5, 32)
ChargeTrain shape: (4304, 5, 14)
SOCTrain shape: (4304, 1, 1)
AnomalyTrain shape: (4304, 1, 1)

TripVal shape: (922, 5, 32)
ChargeVal shape: (922, 5, 14)
SOCVal shape: (922, 1, 1)
AnomalyVal shape: (922, 1, 1)


#### Model Training

In [468]:
from keras.losses import Huber

Optim = Adam(learning_rate=0.00001)

FCModel = Sequential()

FCModel.add(Input(shape=TripTrain.shape[1:]))
FCModel.add(LSTM(128, return_sequences=True, 
                 dropout=0.2, 
                 recurrent_dropout=0.2))
FCModel.add(Dense(64, activation='relu'))

FCModel.add(LSTM(64,
                 return_sequences=True,
                 dropout=0.2,
                 recurrent_dropout=0.2))
FCModel.add(BatchNormalization())


FCModel.add(Dense(64, activation='relu'))
FCModel.add(Dropout(0.4))

FCModel.add(Dense(1, activation='relu'))

FCModel.compile(optimizer=Optim,
                metrics=['mae'],
                loss=Huber(delta=1.0))

FCModel.summary()

Model: "sequential_21"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_72 (LSTM)                  │ (None, 5, 128)         │        82,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_98 (Dense)                │ (None, 5, 64)          │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_73 (LSTM)                  │ (None, 5, 64)          │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_18          │ (None, 5, 64)          │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_99 (Dense)                │ (None, 5, 64)          │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 5, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_100 (Dense)               │ (None, 5, 1)           │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 128,193 (500.75 KB)

 Trainable params: 128,065 (500.25 KB)

 Non-trainable params: 128 (512.00 B)

In [469]:
FCHistory = FCModel.fit(TripTrain,
                        SOCTrain,
                        validation_data=(TripVal, SOCVal),
                        batch_size=16,
                        epochs=50,
                        verbose=1
                        )

Epoch 1/50


269/269 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - loss: 0.4514 - mae: 0.8813 - val_loss: 0.4165 - val_mae: 0.8483
Epoch 2/50
269/269 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 0.4236 - mae: 0.8523 - val_loss: 0.4143 - val_mae: 0.8460
Epoch 3/50
269/269 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.4145 - mae: 0.8439 - val_loss: 0.4102 - val_mae: 0.8411
Epoch 4/50
269/269 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.4167 - mae: 0.8430 - val_loss: 0.4073 - val_mae: 0.8371
Epoch 5/50
269/269 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.4092 - mae: 0.8335 - val_loss: 0.4058 - val_mae: 0.8346
Epoch 6/50
269/269 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.4010 - mae: 0.8265 - val_loss: 0.4043 - val_mae: 0.8321
Epoch 7/50
269/269 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.3971 - mae: 0.8222 - val_loss: 0.4026 - val_mae: 0.8292
Epoch 8/50
269/269 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 0.3963 - mae: 0.8213 - val_loss: 0.4027 - val_mae: 0.8291
Epoch 9/50
269/269 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - loss: 0.3950 

In [357]:
TripFeatures = TripTrain.shape[2]
ChargeFeatures = ChargeTrain.shape[2]

TripInput = Input(shape=(SequenceLength, TripFeatures), name='TripInput')
ChargeInput = Input(shape=(SequenceLength, ChargeFeatures), name='ChargeInput')


TripDense = Dense(64, activation='relu')(TripInput)
ChargeDense = Dense(64, activation='relu')(ChargeInput)

Shared = concatenate([TripDense, ChargeDense])
SharedDense = Dense(128, activation='relu')(Shared)

LSTMLayer = LSTM(64, return_sequences=True)(SharedDense)
LSTMOutput = Dense(1, activation='linear', name='SOCOutput')(LSTMLayer)

Encoder = LSTM(64, activation='relu', return_sequences=False)(SharedDense)
Repeat = RepeatVector(SequenceLength)(Encoder)
Decoder = LSTM(64, activation='relu', return_sequences=True)(Repeat)
AutoEncoderOutput = TimeDistributed(Dense(ChargeFeatures), name='AnomalyOutput')(Decoder)

CombinedModel =  Model(inputs=[TripInput, ChargeInput],
              outputs=[LSTMOutput, AutoEncoderOutput])


CombinedModel.compile(
    optimizer = 'adam',
    loss = {
        'SOCOutput': 'mse',
        'AnomalyOutput': 'mse',
    },
    metrics = {
        'SOCOutput': 'mae',
        'AnomalyOutput': 'mae'
    }
)

CombinedModel.summary()

Model: "functional_12"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ TripInput           │ (None, 5, 32)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ChargeInput         │ (None, 5, 14)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_48 (Dense)    │ (None, 5, 64)     │      2,112 │ TripInput[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_49 (Dense)    │ (None, 5, 64)     │        960 │ ChargeInput[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_12      │ (None, 5, 128)    │          0 │ dense_48[0][0],   │
│ (Concatenate)       │                   │            │ dense_49[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_50 (Dense)    │ (None, 5, 128)    │     16,512 │ concatenate_12[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_37 (LSTM)      │ (None, 64)        │     49,408 │ dense_50[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_vector_12    │ (None, 5, 64)     │          0 │ lstm_37[0][0]     │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_36 (LSTM)      │ (None, 5, 64)     │     49,408 │ dense_50[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_38 (LSTM)      │ (None, 5, 64)     │     33,024 │ repeat_vector_12… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ SOCOutput (Dense)   │ (None, 5, 1)      │         65 │ lstm_36[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ AnomalyOutput       │ (None, 5, 14)     │        910 │ lstm_38[0][0]     │
│ (TimeDistributed)   │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 152,399 (595.31 KB)

 Trainable params: 152,399 (595.31 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
History = CombinedModel.fit(
    x={"TripInput": TripTrain, "ChargeInput": ChargeTrain},
    y={"SOCOutput": SOCTrain, "AnomalyOutput": AnomalyTrain},
    validation_data=(
        {"TripInput": TripVal, "ChargeInput": ChargeVal},
        {"SOCOutput": SOCVal, "AnomalyOutput": AnomalyVal}
    ),
    batch_size=64,
    epochs=30,
    verbose=1
)

Epoch 1/75
308/308 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - AnomalyOutput_loss: 3.8233 - AnomalyOutput_mae: 1.5219 - SOCOutput_loss: 78.8916 - SOCOutput_mae: 5.8259 - loss: 82.7155 - val_AnomalyOutput_loss: 183.7495 - val_AnomalyOutput_mae: 10.3161 - val_SOCOutput_loss: 601.3995 - val_SOCOutput_mae: 17.4850 - val_loss: 782.1373
Epoch 2/75
308/308 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - AnomalyOutput_loss: 3.8206 - AnomalyOutput_mae: 1.4846 - SOCOutput_loss: 73.8583 - SOCOutput_mae: 5.7373 - loss: 77.6790 - val_AnomalyOutput_loss: 184.7408 - val_AnomalyOutput_mae: 10.3633 - val_SOCOutput_loss: 611.4686 - val_SOCOutput_mae: 17.6745 - val_loss: 793.9921
Epoch 3/75
308/308 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - AnomalyOutput_loss: 5.0927 - AnomalyOutput_mae: 1.7446 - SOCOutput_loss: 78.4151 - SOCOutput_mae: 5.8745 - loss: 83.5082 - val_AnomalyOutput_loss: 183.2962 - val_AnomalyOutput_mae: 10.4416 - val_SOCOutput_loss: 586.9986 - val_SOCOutput_mae: 17.3552 - val_loss: 774.9686
Epoch 4/75
308/308 ━━━━━━━━━